Modify the location to open up a new ckpt file.

In [ ]:
location = ''

#Add name: shape
#EXAMPLE: = {"hi":shape[2,2], "I'm":shape[5], "an":shape[5,5], "example":shape[10,10,5]}
varToShapeMapping = {}

In [1]:
import re
import tensorflow as tf

def loadCheckpoint():
    file_location = addCkpt()
    #Extra layer of abstraction, loadCkpt strictly loads a ckpt if it exists
    try:
        loadCkpt(file_location)
        print('Successfully loaded checkpoint at: ' + location)
    except:
        print('Failed to load checkpoint.')

def addCkpt():
    # If the end of the file is ckpt then we don't need to append it.
    if re.search('\.ckpt$', location):
        location = location
    else:
        location = location + '.ckpt'

def loadCkpt(file_location):
    tf.reset_default_graph()
    for var, varShape in varToShapeMapping.iteritems():
        test = tf.get_variable(var, varShape)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, file_location)
        print('Model restored!')
        print('But the variables aren\'t setup.')